# EE 467 Lab 5: Detecting Credit Card Fraud with Decision Tree

In lab 5 and lab 6, we will go through a famous machine learning contest on Kaggle-**detecting fraud credit card transactions**, and learn about **decision tree, random forest and ensemble learning algorithms** in the meantime. Today, our task is to analyze and pre-process the credit card transactions dataset to resolve its inherent issues. We will then train a decision tree classifier on the transform features to identify fraud credit card transactions.

## Task Description
### Background

Fraudulent credit card transactions create major losses for both cardholders and financial institutions. According to the Nilson Report, worldwide payment card fraud losses were \$33.83B in 2023 and \$33.41B in 2024.

### Challenges in Detection

1. Uncommon: Fraud transactions are rare compared to normal transactions, hence there are few samples where we can learn from to identify these transactions. We will see in the later part of this lab that less than 0.2% of the transactions are in the fraud class.
2. Concealed: Fraudsters will try their best to blend in and conceal their activities.
3. Change over time: Patterns of fraud transactions change over time so that fraudsters could avoid getting caught.

### Evaluate Machine Learning-based Fraud Detection

Traditionally, rules-based expert systems are deployed to catch fraud transactions. While they could do an excellent jon to uncover existing fraud patterns, they are not effective for new and covert fraud schemes. Recently, financial institutions start to apply machine learning algorithms to credit card fraud identification because of their flexibility and adaptation to new fraud patterns. However, when compared to regular machine learning tasks, there are a few extra requirements for evaluation of the fraud transaction classifier:

* Instead of looking at the overall accuracy, we are more interested in the precision and recall, as the credit card transactions dataset is highly unbalanced. A deterministic classifier that always marks transactions as non-fraud could achieve over 99.8% of overall accuracy, but such a classifier is meaningless in terms of identifying fraud because its precision and recall in the fraud class are both zero.
* Besides, we also want to reduce false positives because in that case normal transactions are also blocked, which can be annoying for benign credit card users.

## Lab Dependencies

Apart from `numpy`, `pandas`, `scikit-learn` (`sklearn`) and `matplotlib`, we need to install two more external libraries for this lab: `seaborn` and `imbalanced-learn` (`imblearn`).

In [ ]:
%pip install numpy pandas scikit-learn matplotlib seaborn imbalanced-learn

In [ ]:
!tar -xJf credit-card.tar.xz #<--- Use this to unzip the dataset folder in Colab

## Feature Analysis

Before building machine learning classification models, we need to know more about the characteristics of the credit card transactions dataset. As usual, we will begin with loading the credit card dataset into memory and preview its format and statistics:

In [ ]:
import pandas as pd

# Read the credit card dataset into memory
df = pd.read_csv("creditcard.csv")

# Preview the first few records
df.head(10)

In [ ]:
# Preview the statistics of each column
df.describe()

The preview results indicate that there are 30 features for each credit card transaction (record), of which 28 of them (`V1` to `V28`) are **opaque features obtained from Principle Component Analysis (PCA)**. The `Time` and `Amount` features indicate the **relative time and amount** of each transaction and none of them is normalized (re-scaled). The `Class` column serves as the **labels of all transactions**, and we are interested in the categories and distributions of labels:

In [ ]:
import matplotlib.pyplot as plt

# Distribution of transaction classes (fraud / non-fraud)
cat_dist = df["Class"].value_counts()
print("Frequencies of classes:\n", cat_dist, "\n")

# Ratio of classes
print("Ratio of classes:\n", cat_dist/len(df))

# Plot the distribution of classes
plt.xlabel("Class")
plt.ylabel("# of Transactions")
cat_dist.plot(kind="bar", title="Distribution of classes", rot=0)

Apparently, the dataset is **highly imbalanced** in terms of class distributions, as it contains only **492 fraud cases** out of **284,807 total cases**, or **0.172% of fraud cases**. This can be justified by the low frequent and covert nature of fraudulent transactions. In the later part of this lab we will see that special measures must be used to deal with this data imbalance issue.

Next, we will look into each kind of feature individually and try to reveal patterns in credit card transactions.

### Opaque Features (`V1` to `V28`)

The opaque features (`V1` to `V28`) are obtained by reducing the dimensions of raw features with PCA. The raw transaction features of credit card transactions are not provided because of privacy issues, and we have no idea about the meaning of these opaque features. In this case, one approach is to **perform PCA again on the opaque features**, reducing them to three dimensions and then **visualizing them on 2D and 3D plots** (Mathmatically this is equivalent to directly reducing raw features to three dimensions with a single PCA transformation):

In [ ]:
from sklearn.decomposition import IncrementalPCA

import lab_6_util

with lab_6_util.timeit("Visualizing opaque features to 3D"):
    # Reduce opaque features to three dimensions
    opaque_features_3d = IncrementalPCA(n_components=3).fit_transform(df.loc[:, "V1":"V28"])
    # Visualize three-dimensional opaque features
    lab_6_util.visualize_samples(opaque_features_3d, df["Class"], "3D opaque features")

The visualization plot shows that a significant portion of fraud transactions can be separated from normal transactions in the 3D space. However, to achieve better classification performance, we should incorporate additional features beyond just the opaque PCA features.

### Transaction Time

To reveal potential patterns in transaction time, we plot its distribution with `seaborn`, a library for statistical data visualization:

In [ ]:
import seaborn as sns

# Convert transaction time from seconds to hours
time_df = df["Time"] / (60 * 60)

# Plot the distribution of transaction time
sns.histplot(time_df, color="r", kde=True, stat="density")
# Set plot title
plt.title("Distribution of transaction time")
# Set plot range
plt.xlim((time_df.min(), time_df.max()))

# Show plot
plt.show()

The plot indicates that all transactions in the dataset were collected during a **two-day period** (maximum time is 48 hours). Most transactions occur during a continuous 16-hour period in a single day, corresponding to when people are typically awake. Note that time 0 is a relative timestamp and does not correspond to any specific clock time.

Now let's examine the distribution **separately for fraud and non-fraud transactions** to identify any temporal patterns:

In [ ]:
import numpy as np

# Fraud and non-fraud transactions
df_nonfraud = df[df["Class"]==0]
df_fraud = df[df["Class"]==1]

# X-axis of the transaction time plot
bins = np.arange(48)
# Transaction time for fraud and non-fraud transactions (in hours)
time_nonfraud = df_nonfraud["Time"]/(60*60)
time_fraud = df_fraud["Time"]/(60*60)

## [ TODO ]
# Plot transaction time distribution of fraud and non-fraud transactions SEPARATELY
# (Caution: plot the percentiles (NOT amount) for each range)
# (Hint: use `plt.hist` to plot; set fraud and non-fraud series to differnent colors and increase transparency)
raise NotImplementedError

plt.show()

We can observe from the plot that the proportion of non-fraud transactions decreases from hour 1 to 8 and from hour 24 to 32, while the proportion of fraud transactions increases during these periods. This suggests that **fraud transactions tend to occur more frequently during night hours**.

### Transaction Amount

Similar to the transaction time, let's also plot the distribution of transaction amount:

In [ ]:
# Amount of all transactions
amount_df = df["Amount"]

# Plot the distribution of transaction amount
sns.histplot(amount_df, color="r", kde=True, stat="density")
# Set plot title
plt.title("Distribution of transaction amount")
# Set plot range
plt.xlim((0, amount_df.max()))

plt.show()

The distribution of transaction amount exhibits a classic **long-tail distribution**: most transactions are of low value, but there are outliers with amounts several times the mean. This characteristic can be seen in the statistics of non-fraud, fraud, and all transactions:

In [ ]:
# Distribution percentiles
dist_pc = np.arange(0.1, 1, 0.1)

# Amount of non-fraud and fraud transactions
amount_nonfraud = df_nonfraud["Amount"]
amount_fraud = df_fraud["Amount"]

# Statistics of amount of non-fraud, fraud and all transactions
amount_stats = pd.DataFrame.from_dict({
    "Non-Fraud": amount_nonfraud.describe(percentiles=dist_pc),
    "Fraud": amount_fraud.describe(percentiles=dist_pc),
    "All": amount_df.describe(percentiles=dist_pc)
})

amount_stats

The average fraud transaction amount is $122 USD, while non-fraud transactions average $88 USD. The distributions are similar for the bottom 60% of transactions, but **fraud transactions have higher values in the top 40%**. We can further demonstrate this pattern by plotting the distribution of high-value transactions:

In [ ]:
# X-axis of the transaction amount plot
bins = np.arange(200, 2500, 25)

# Plot high-value fraud and non-fraud transactions ($200 to $2000)
plt.hist(amount_nonfraud, bins, density=True, label="Non-Fraud", color="b", alpha=0.5)
plt.hist(amount_fraud, bins, density=True, label="Fraud", color="r", alpha=0.5)
# Set plot title
plt.title("Distribution of amount of high-value transactions")
# Show legend
plt.legend(fancybox=True)

plt.show()

We can conclude from this plot that fraud transactions tend to occur more frequently in the high-value "tail" of the transaction distribution. However, the amount feature alone would still be insufficient for accurate fraud detection.

## Feature Scaling

With the analysis of different feature types complete, we now move to the **feature scaling** step. The opaque features (`V1` to `V28`) are generated by PCA and are already normalized (measured in units of standard deviations), so they require no additional preprocessing. However, the Transaction Time and Amount features are raw values that need rescaling:

- **`StandardScaler`** could be applied to transaction time (after taking modulo 24 to get hour-of-day), but it is inappropriate for transaction amount because the amount distribution is not normal.
- **`MinMaxScaler`** is also inappropriate for transaction amount because the long-tail distribution contains many outliers that would distort the scaling.

Instead, we use **[`RobustScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)**, which performs a linear transformation using the interquartile range (IQR) as the scale factor, making it robust against outliers:

$$x_{\text{scaled}} = \frac{x - Q_{25}(x)}{Q_{75}(x) - Q_{25}(x)}$$

This formula uses the 25th percentile ($Q_{25}$) as the baseline and the difference between 75th and 25th percentiles as the scale, ensuring that outliers have minimal impact on the transformation.

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler

# Copy original dataset
df_scaled = df.copy()

## [ TODO ]
# 1) Divide transaction time by 24 (hours in a day)
# 2) Rescale and override transaction time column with `StandardScaler`
# 3) Rescale and override transaction amount column with `RobustScaler`
#    (Hint: refer to lab 5 and API reference for usage of feature scalers)
raise NotImplementedError

# Review scaling results
df_scaled[["Amount", "Time"]].head(5)

## Training and Evaluation

We now begin the training stage by performing a **train-test split** on the scaled dataset. Since fraud transactions are rare, we set the **test set size to 40%** to ensure sufficient fraud samples are included in the test set for reliable evaluation:

In [ ]:
from sklearn.model_selection import train_test_split

# Get feature and label values from original dataset
feat_all = df_scaled.drop(["Class"], axis=1).values
y_all = df_scaled["Class"].values

# Split samples into training and test sets
feat_train, feat_test, y_train, y_test = train_test_split(
    feat_all, y_all, test_size=0.4, random_state=0
)

### Data Resampling with SMOTE

Our earlier analysis showed that the credit card fraud dataset is highly imbalanced: over 99.8% of transactions are non-fraud. To address this, we apply **data resampling** to adjust the class distribution before training. We can either **over-sample** the fraud class or **under-sample** the non-fraud class; here we use **Synthetic Minority Over-sampling TEchnique (SMOTE)**.

SMOTE synthesizes new minority samples through $k$-**nearest neighbors**:

1. Randomly select a fraud sample and find its $k$ nearest neighbors.
2. Randomly select one of these neighbors.
3. Create a synthetic sample via linear interpolation between the two selected samples.

An extension of SMOTE, called **borderline SMOTE**, enhances the original technique by first learning an **decision boundary** between the majority and minority classes and then applying SMOTE only to minor samples that are adjacent to the decision boundary. Here we will try both original and borderline SMOTE for the fraud class:

In [ ]:
from imblearn.over_sampling import SMOTE, BorderlineSMOTE

# SMOTE over-sampling
smote = SMOTE(random_state=0)
# Borderline SMOTE over-sampling
borderline_smote = BorderlineSMOTE(random_state=0)

### Decision Tree Classifier

A **decision tree** is a machine learning model that uses a hierarchical, tree-like structure for making predictions. To make a prediction with a decision tree:

1. Start at the root node and evaluate the associated condition
2. Follow the appropriate branch based on the condition result
3. Recursively evaluate conditions at each sub-node, moving deeper into the tree
4. Continue until reaching a leaf node, which provides the final prediction

Decision trees are versatile and work for both classification and regression problems. Below are visualizations showing a simple decision tree and its decision boundary:


Now, we will build and train three [`DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html). The first one will be trained directly on the original samples, while the second and third one will be trained with the fraud class over-sampled using original and borderline SMOTE respectively:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline

## [ TODO ]
# 1) Train a decision tree classifier `dt` on original samples.
# 2) Build a ML pipeline `dt_smote` with SMOTE over-sampling and a decision tree classifier,
#    then train the whole pipeline.
# 3) Build a ML pipeline `dt_borderline_smote` with borderline SMOTE over-sampling and
#    a decision tree classifier, then train the whole pipeline
raise NotImplementedError

## Evaluation

With the three decision tree classifiers trained, we now evaluate their performance on the test set using the **confusion matrix** and **classification report**, which includes the metrics we encountered before:

- **Precision**: Of the transactions flagged as fraud, how many actually were fraud?
- **Recall**: Of all fraud transactions in the test set, how many did we correctly identify?
- **F1-score**: The harmonic mean of precision and recall, balancing both metrics
- **Accuracy**: Overall correctness, though less useful for imbalanced datasets

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

def plot_m2m_curve(ax, xs, ys, x_metric, y_metric, title):
    """Helper function for plotting metric-to-metric curve."""
    # Plot metric-to-metric curve
    ax.plot(xs, ys)

    # Make sub-plot square
    ax.set_aspect("equal")
    # Set axes labels
    ax.set_xlabel(x_metric)
    ax.set_ylabel(y_metric)
    # Set title
    ax.set_title(title)

def evaluate_model(model, name, feat_test, y_test):
    """ Evaluate a classification model on the test set, then print and plot metrics. """
    # Make prediction from features
    pred_test = model.predict(feat_test)

    print(f"\n[ Evaluation result for {name} ]")
    # Print classification report
    print("Classification report:")
    print(classification_report(y_test, pred_test))

    # Print confusion matrix
    print("Confusion matrix:")
    print(confusion_matrix(y_test, pred_test), "\n")

In [ ]:
# Evaluate all three trained models
evaluate_model(dt, "DT (no resampling)", feat_test, y_test)
evaluate_model(dt_smote, "DT with SMOTE", feat_test, y_test)
evaluate_model(dt_borderline_smote, "DT with borderline SMOTE", feat_test, y_test)

From the evaluation results, we observe that **over-sampling the fraud class does not improve precision, recall, or F1-score**. In fact, performance degradation is observed when original SMOTE over-sampling is applied. This suggests that simple decision trees may be limited for this task. In the next lab, we will explore ensemble learning methods and random forests to improve classification performance.

## References

1. Kaggle credit card fraud dataset: https://www.kaggle.com/mlg-ulb/creditcardfraud
2. Seaborn API reference: https://seaborn.pydata.org/api.html
3. SMOTE over-sampling: https://en.wikipedia.org/wiki/Oversampling_and_undersampling_in_data_analysis#SMOTE
4. SMOTE for imbalance data classification: https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/
5. `imbalanced-learn` over-sampling API reference: https://imbalanced-learn.readthedocs.io/en/stable/api.html#module-imblearn.over_sampling
6. Receiver Operating Characteristic (ROC) curve: https://en.wikipedia.org/wiki/Receiver_operating_characteristic
7. ROC Curves and Precision-Recall Curves for Imbalanced Classification: https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-imbalanced-classification/
8. Decision tree: https://en.wikipedia.org/wiki/Decision_tree
9. Decision tree learning: https://en.wikipedia.org/wiki/Decision_tree_learning
10. Decision tree algorithm with example: https://www.youtube.com/watch?v=RmajweUFKvM